# Search and Visualize Tropical Cyclones from IBTrACS

This notebook allows you to search for tropical cyclones in the IBTrACS (International Best Track Archive for Climate Stewardship) collection by name and visualize their tracks on an interactive map.

We'll use the STAC API at https://montandon-eoapi-stage.ifrc.org/stac/collections/ibtracs-events to search for cyclones by name and retrieve their track data.

## 1. Import Libraries

In [1]:
# Basic libraries
import json
from datetime import datetime

# Visualization libraries
import folium
import pandas as pd
from IPython.display import display, HTML

# STAC libraries
import pystac
import pystac_client
from pystac_monty.extension import MontyExtension

# Import STAC helper functions
import sys
sys.path.append('.')
from stac_helpers import check_stac_api_availability

## 2. Connect to STAC API and Define Search Functions

In [3]:
# Define the STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

if not api_available:
    print("STAC API is not available. Please check the URL or try again later.")
else:
    # Connect to the STAC API using pystac-client
    try:
        catalog = pystac_client.Client.open(stac_api_url)
        print(f"Successfully connected to STAC API at {stac_api_url}")
        
        # Check if the ibtracs-events collection exists
        collections = catalog.get_collections()
        collection_ids = [collection.id for collection in collections]
        
        if "ibtracs-events" in collection_ids:
            print("Found ibtracs-events collection")
        else:
            print("Warning: ibtracs-events collection not found in the catalog")
            
    except Exception as e:
        print(f"Error connecting to STAC API: {e}")
        catalog = None

STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac
Successfully connected to STAC API at https://montandon-eoapi-stage.ifrc.org/stac
Found ibtracs-events collection


In [17]:
def search_cyclones_by_name(name, catalog=catalog, collection_id="ibtracs-events", limit=10):
    """
    Search for tropical cyclones by name in the IBTrACS collection using pystac-client

    Parameters:
    - name: Name of the cyclone to search for (case-insensitive)
    - catalog: pystac_client.Client instance
    - collection_id: ID of the collection to search in
    - limit: Maximum number of results to return

    Returns:
    - List of pystac.Item objects matching the search criteria
    """
    if not catalog:
        print("STAC API client not available")
        return []

    try:
        # Create a search with the specified parameters
        search = catalog.search(
            collections=[collection_id],
            filter={"op": "like", "args": [{"property": "title"}, f"%{name.upper()}%"]},
            limit=limit
        )

        # Execute the search and get the items
        items = list(search.items())
        return items
    except Exception as e:
        print(f"Error searching for cyclones: {e}")
        return []

In [5]:
def get_cyclone_by_id(cyclone_id, catalog=catalog, collection_id="ibtracs-events"):
    """
    Get a specific cyclone by its ID using pystac-client
    
    Parameters:
    - cyclone_id: ID of the cyclone to retrieve
    - catalog: pystac_client.Client instance
    - collection_id: ID of the collection containing the cyclone
    
    Returns:
    - pystac.Item representing the cyclone, or None if not found
    """
    if not catalog:
        print("STAC API client not available")
        return None
    
    try:
        # Get the collection
        collection = catalog.get_collection(collection_id)
        
        # Get the item by ID
        item = collection.get_item(cyclone_id)
        return item
    except Exception as e:
        print(f"Error retrieving cyclone {cyclone_id}: {e}")
        return None

## 3. Define Visualization Functions

In [6]:
def create_cyclone_map(cyclone_item):
    """
    Create an interactive map showing the track of a tropical cyclone
    
    Parameters:
    - cyclone_item: pystac.Item representing a tropical cyclone
    
    Returns:
    - Folium map object
    """
    if not cyclone_item:
        print("No cyclone data provided")
        return None
    
    # Extract the cyclone track coordinates
    geometry = cyclone_item.geometry
    if geometry.get("type") != "LineString":
        print(f"Unexpected geometry type: {geometry.get('type')}")
        return None
    
    coordinates = geometry.get("coordinates", [])
    if not coordinates:
        print("No coordinates found in cyclone data")
        return None
    
    # Get the cyclone properties
    title = cyclone_item.properties.get("title", "Unknown Cyclone")
    description = cyclone_item.properties.get("description", "")
    start_date = cyclone_item.properties.get("start_datetime", "")
    end_date = cyclone_item.properties.get("end_datetime", "")
    
    # Format dates for display
    if start_date:
        start_date = datetime.fromisoformat(start_date.replace('Z', '+00:00')).strftime('%Y-%m-%d')
    if end_date:
        end_date = datetime.fromisoformat(end_date.replace('Z', '+00:00')).strftime('%Y-%m-%d')
    
    # Create a map centered on the middle of the cyclone track
    mid_point_index = len(coordinates) // 2
    mid_point = coordinates[mid_point_index]
    
    # Create the map
    m = folium.Map(location=[mid_point[1], mid_point[0]], zoom_start=4)
    
    # Add the cyclone track as a polyline
    # Convert coordinates from [lon, lat] to [lat, lon] for folium
    track_points = [[coord[1], coord[0]] for coord in coordinates]
    
    # Add the track line with a gradient color to show progression
    folium.PolyLine(
        track_points,
        color='blue',
        weight=3,
        opacity=0.8,
        tooltip=f"{title} ({start_date} to {end_date})"
    ).add_to(m)
    
    # Add markers for the start and end points
    folium.Marker(
        [track_points[0][0], track_points[0][1]],
        popup=f"Start: {start_date}",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)
    
    folium.Marker(
        [track_points[-1][0], track_points[-1][1]],
        popup=f"End: {end_date}",
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)
    
    # Add markers at regular intervals to show progression
    step = max(1, len(track_points) // 10)  # Add up to 10 markers along the track
    for i in range(step, len(track_points) - 1, step):
        folium.CircleMarker(
            track_points[i],
            radius=3,
            color='orange',
            fill=True,
            fill_color='orange',
            fill_opacity=0.8
        ).add_to(m)
    
    return m

In [7]:
def display_cyclone_info(cyclone_item):
    """
    Display information about a tropical cyclone using pystac and pystac-monty helpers
    
    Parameters:
    - cyclone_item: pystac.Item representing a tropical cyclone
    """
    if not cyclone_item:
        print("No cyclone data provided")
        return
    
    # Extract basic information
    title = cyclone_item.properties.get("title", "Unknown Cyclone")
    description = cyclone_item.properties.get("description", "No description available")
    start_date = cyclone_item.properties.get("start_datetime", "Unknown")
    end_date = cyclone_item.properties.get("end_datetime", "Unknown")
    
    # Format dates for display
    if start_date and start_date != "Unknown":
        start_date = datetime.fromisoformat(start_date.replace('Z', '+00:00')).strftime('%Y-%m-%d')
    if end_date and end_date != "Unknown":
        end_date = datetime.fromisoformat(end_date.replace('Z', '+00:00')).strftime('%Y-%m-%d')
    
    # Extract Monty extension properties using the MontyExtension helper
    monty_ext = MontyExtension.ext(cyclone_item)
    country_codes = monty_ext.country_codes or []
    hazard_codes = monty_ext.hazard_codes or []
    correlation_id = monty_ext.correlation_id or "Unknown"
    
    # Display the information
    print(f"\n{title}")
    print("=" * len(title))
    print(f"\nDescription: {description}")
    print(f"\nDuration: {start_date} to {end_date}")
    print(f"\nAffected Countries: {', '.join(country_codes)}")
    print(f"\nHazard Codes: {', '.join(hazard_codes)}")
    print(f"\nCorrelation ID: {correlation_id}")
    
    # Display any additional keywords
    keywords = cyclone_item.properties.get("keywords", [])
    if keywords:
        print(f"\nKeywords: {', '.join(keywords)}")

## 4. Search and Visualize Cyclones

In [8]:
def search_and_display_cyclone(cyclone_name):
    """
    Search for a cyclone by name and display its information and track
    
    Parameters:
    - cyclone_name: Name of the cyclone to search for
    """
    # Search for cyclones matching the name
    search_results = search_cyclones_by_name(cyclone_name)
    
    if not search_results:
        print(f"No cyclones found matching '{cyclone_name}'")
        return
    
    # Display the search results
    print(f"Found {len(search_results)} cyclones matching '{cyclone_name}':\n")
    
    # Create a DataFrame to display the results
    results_df = pd.DataFrame([
        {
            "ID": item.id,
            "Title": item.properties.get("title", "Unknown"),
            "Start Date": item.properties.get("start_datetime", "Unknown"),
            "End Date": item.properties.get("end_datetime", "Unknown"),
            "Countries": ", ".join(MontyExtension.ext(item).country_codes or [])
        }
        for item in search_results
    ])
    
    # Format dates for display
    for col in ["Start Date", "End Date"]:
        results_df[col] = results_df[col].apply(
            lambda x: datetime.fromisoformat(x.replace('Z', '+00:00')).strftime('%Y-%m-%d') if x != "Unknown" else x
        )
    
    display(results_df)
    
    # If there's only one result, display it automatically
    if len(search_results) == 1:
        cyclone_item = search_results[0]
        display_cyclone_info(cyclone_item)
        m = create_cyclone_map(cyclone_item)
        if m:
            display(m)
    else:
        # If there are multiple results, let the user select one
        print("\nEnter the ID of the cyclone you want to visualize in the next cell.")

In [9]:
def display_cyclone_by_id(cyclone_id):
    """
    Display information and track for a specific cyclone by ID
    
    Parameters:
    - cyclone_id: ID of the cyclone to display
    """
    # Get the cyclone data
    cyclone_item = get_cyclone_by_id(cyclone_id)
    
    if not cyclone_item:
        print(f"No cyclone found with ID '{cyclone_id}'")
        return
    
    # Display the cyclone information
    display_cyclone_info(cyclone_item)
    
    # Create and display the map
    m = create_cyclone_map(cyclone_item)
    if m:
        display(m)

## 5. Search for a Cyclone

Enter the name of a tropical cyclone to search for (e.g., "Beryl", "Katrina", "Maria"):

In [18]:
# Example: Search for Hurricane Beryl
cyclone_name = "Garance"  # Change this to search for a different cyclone
search_and_display_cyclone(cyclone_name)

Found 1 cyclones matching 'Garance':



,ID,Title,Start Date,End Date,Countries
0,2025057S18052,Tropical Cyclone GARANCE,2025-02-25,2025-03-02,UNK



Tropical Cyclone GARANCE

Description: Tropical Cyclone GARANCE (2025) in the South Indian basin. Maximum intensity: Category 3 hurricane with 120 mph (105.0 knots) winds and minimum pressure of 953.0 mb.

Duration: 2025-02-25 to 2025-03-02

Affected Countries: UNK

Hazard Codes: MH0057, nat-met-sto-tro, TC

Correlation ID: 20250225T120000-UNK-NAT-MET-STO-TRO-001-GCDB

Keywords: tropical cyclone, hurricane, GARANCE, 2025, South Indian
